In [1]:
import torch
import models.pointnet2_classification as ptnet
from models.cirtorch_network import init_network, extract_vectors
import torch_geometric
from torch_geometric.data import Data
from models.cirtorch_utils.genericdataset import PointCloudImagesFromList, PCDataLoader
from dataset_loaders.txt_to_db import get_images, get_points
from torchvision import transforms
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from torch_geometric.data import DataLoader

In [2]:
pointnet = ptnet.NetAachen()

In [3]:
test_sample = Data(pos=torch.Tensor(10, 3), batch=torch.LongTensor([0]*10))

In [4]:
test_result = pointnet(test_sample)

In [5]:
print(test_result.size())

torch.Size([1, 512])


In [6]:
#params = {'mean', [], 'std': []}
cir = init_network({'architecture' : 'resnet34'})

In [7]:
fd = cir(torch.Tensor(1, 3, 256, 256))

In [8]:
print(fd.size())

torch.Size([512, 1])


In [9]:
images = get_images()
points3d = get_points()

In [10]:
# set up the transform
normalize = transforms.Normalize(
   mean=cir.meta['mean'],
   std=cir.meta['std']
)
transform = transforms.Compose([
    transforms.CenterCrop(1024),
    transforms.ToTensor(),
    normalize
])

In [11]:
sampler = PointCloudImagesFromList('data/AachenDayNight/images_upright', images, points3d, imsize=1024, transform=transform, triplet=True, min_num_points=100)
print('{} items in dataset'.format(len(sampler)))

4278 items in dataset


In [12]:
dl = PCDataLoader(sampler, batch_size=2)

In [15]:
for i, d in enumerate(dl):
    print(i)
    print(d[0].size())
    print(d[1])
    print(d[2])
    break
t1 = cir(d[0])
t2 = pointnet(d[1])
print(t1.size())
print(t2.size())

0
torch.Size([2, 3, 1024, 1024])
Batch(batch=[7763], pos=[7763, 3])
Batch(batch=[7230], pos=[7230, 3])
torch.Size([512, 2])
torch.Size([2, 512])


In [ ]:
test_id = 2535
test_img, test_pts, test_triplet = sampler[test_id]

In [ ]:
print(test_pts.size())

In [ ]:
plt.imshow(test_img.transpose(0, -1).transpose(0,1).numpy()*cir.meta['std']+cir.meta['mean'])
plt.axis('off')
plt.show()

In [ ]:
tp = test_pts.numpy()
tt = test_triplet.numpy()
tpt = np.concatenate([tp, tt])
mid, rng = tpt.mean(axis=0), 2*tpt.std(axis=0)
print(mid)
print(rng)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tp[:,0], tp[:,2], tp[:,1], color='r')
ax.scatter(tt[:,0], tt[:,2], tt[:,1], color='g')
ax.set_xlim3d(mid[0]-rng[0], mid[0]+rng[0])
ax.set_zlim3d(mid[1]-rng[1], mid[1]+rng[1])
ax.set_ylim3d(mid[2]-rng[2], mid[2]+rng[2])
plt.show()

In [ ]:
image_ids = {i : k for i, k in enumerate(images.keys())}

In [ ]:
plt.imshow(plt.imread('data/AachenDayNight/images_upright/'+images[image_ids[test_id]].name))
plt.show()

In [ ]:
pt_ids = images[image_ids[test_id]].point3D_ids[images[image_ids[test_id]].point3D_ids > 0]
pts = torch.stack([torch.from_numpy(points3d[i].xyz) for i in pt_ids]).numpy()
print(np.linalg.norm(pts - test_pts.numpy()))

In [ ]:
import random

In [ ]:
random.seed(0)

In [ ]:
random.choice(image_ids)

In [ ]:
i = 0
while True:
    trp_idx = random.choice(image_ids)
    print(trp_idx)
    valid_trp = images[trp_idx].point3D_ids > 0
    pt_ids_trp = images[trp_idx].point3D_ids[valid_trp]
    shared = np.intersect1d(pt_ids, pt_ids_trp, assume_unique=True)
    print(shared.shape)
    print(pt_ids.shape[0]*0.01)
    if shared.shape[0] < pt_ids.shape[0] * 0.01:
        break
    i += 1
print(i)

In [ ]:
data_to_ptnet = lambda x: Data(pos=x.float(), batch=torch.LongTensor([0]*x.size(0)))

In [ ]:
x = pointnet(data_to_ptnet(test_pts))
y = pointnet(data_to_ptnet(test_triplet))
z = cir(test_img.unsqueeze(0))

In [ ]:
print(x.size())
print(y.size())
print(z.size())

In [ ]:
for i, x in enumerate(sampler):
    if (i % 1000) == 0:
        print(i)
print(i)